# Chapter 20: Recursion and Backtracking

> *"To understand recursion, you must first understand recursion."* — Anonymous

---

## 20.1 Introduction to Recursion

**Recursion** is a problem-solving technique where a function calls itself to solve smaller instances of the same problem. It is a fundamental concept in computer science, underlying divide-and-conquer algorithms, tree traversals, and backtracking.

### 20.1.1 Why Recursion Matters

```
┌─────────────────────────────────────────────────────────────────────┐
│                    IMPORTANCE OF RECURSION                           │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. NATURAL FIT: Many problems have recursive structure             │
│     (trees, graphs, mathematical definitions)                       │
│                                                                      │
│  2. ELEGANCE: Recursive solutions are often concise and expressive  │
│                                                                      │
│  3. FOUNDATION: Recursion is essential for backtracking, divide-    │
│     and-conquer, dynamic programming, and tree/graph algorithms     │
│                                                                      │
│  4. INDUCTION: Reasoning about recursion parallels mathematical     │
│     induction, aiding correctness proofs                            │
│                                                                      │
│  5. FUNCTIONAL PROGRAMMING: Many functional languages rely on       │
│     recursion instead of loops                                       │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### 20.1.2 Recursion: Key Components

A recursive function typically consists of:

1. **Base case(s):** The simplest instance that can be solved directly without recursion.
2. **Recursive case(s):** Break the problem into smaller subproblems and call the function recursively.
3. **Progress toward base case:** Ensure each recursive call moves closer to a base case.

### 20.1.3 Example: Factorial

```python
def factorial(n):
    """Compute n! recursively."""
    # Base case
    if n <= 1:
        return 1
    # Recursive case
    return n * factorial(n - 1)
```

**Call stack for factorial(4):**
```
factorial(4)
  → 4 * factorial(3)
        → 3 * factorial(2)
              → 2 * factorial(1)
                    → 1 (base)
              ← 2
        ← 6
  ← 24
```

### 20.1.4 Recursion vs. Iteration

| Aspect          | Recursion                          | Iteration                          |
|-----------------|------------------------------------|------------------------------------|
| Code clarity    | Often more intuitive               | Can be less clear                  |
| State           | Maintained via call stack          | Maintained in variables             |
| Overhead        | Function call overhead, stack depth| No function call overhead           |
| Memory          | O(depth) stack space               | O(1) extra space                    |
| Suitability     | Tree/divide-and-conquer problems   | Simple loops, linear problems       |

---

## 20.2 Recursion Trees and Stack Frames

A **recursion tree** visualizes recursive calls and their relationships. Each node represents a call, and its children represent recursive subcalls. The tree helps analyze time complexity and understand the flow.

### 20.2.1 Example: Fibonacci

```python
def fib(n):
    if n <= 1:
        return n
    return fib(n-1) + fib(n-2)
```

Recursion tree for fib(4):
```
          fib(4)
         /      \
    fib(3)      fib(2)
    /    \      /    \
 fib(2) fib(1) fib(1) fib(0)
 /    \
fib(1) fib(0)
```

This tree reveals exponential time (O(2ⁿ)) due to repeated subproblems – motivating dynamic programming.

### 20.2.2 Stack Frames

Each recursive call pushes a new **stack frame** containing local variables, parameters, and return address. Excessive recursion can lead to **stack overflow**.

**Python recursion limit:** Typically ~1000; can be increased with `sys.setrecursionlimit()`, but may cause segmentation fault if too deep.

---

## 20.3 Tail Recursion

A recursive function is **tail-recursive** if the recursive call is the last operation performed before returning. Tail recursion can be optimized by compilers to iteration (tail call optimization, TCO), reusing the same stack frame.

### 20.3.1 Example: Factorial (non-tail vs tail)

**Non-tail recursive factorial** (as above) – after recursive call, we still multiply.

**Tail-recursive factorial:**

```python
def factorial_tail(n, accumulator=1):
    if n <= 1:
        return accumulator
    return factorial_tail(n - 1, n * accumulator)
```

Here the recursive call is the final operation; no pending computation after it. Python does **not** perform TCO by default, but the concept is important for languages that do (e.g., Scheme, Scala, some functional languages).

### 20.3.2 Why TCO Matters

- **Prevents stack growth** for deep recursion.
- Enables recursion to be as efficient as iteration.
- Essential for functional programming where loops are not idiomatic.

---

## 20.4 Backtracking Framework

**Backtracking** is a systematic trial-and-error approach to solving constraint satisfaction problems. It incrementally builds candidates and abandons ("backtracks") when a candidate cannot lead to a valid solution.

### 20.4.1 General Backtracking Algorithm

```python
def backtrack(candidate):
    if is_a_solution(candidate):
        process_solution(candidate)
        return
    for next_candidate in generate_candidates(candidate):
        if is_valid(next_candidate):
            make_move(next_candidate)
            backtrack(next_candidate)
            undo_move(next_candidate)
```

This template applies to many problems: N-Queens, Sudoku, subset sum, permutations, etc.

### 20.4.2 State Space Tree

The recursion explores a **state space tree** where each node represents a partial solution. Leaves are complete solutions or dead ends.

```
Root (empty)
├── choice1
│   ├── choice1.1
│   │   └── ...
│   └── choice1.2
├── choice2
...
```

---

## 20.5 Classic Constraint Satisfaction Problems

### 20.5.1 N-Queens

Place N queens on an N×N chessboard so that no two queens attack each other.

```python
def solve_n_queens(n):
    board = [-1] * n  # board[i] = column of queen in row i
    solutions = []
    
    def is_safe(row, col):
        # Check previous rows
        for r in range(row):
            c = board[r]
            if c == col or abs(c - col) == abs(r - row):
                return False
        return True
    
    def backtrack(row):
        if row == n:
            # All queens placed
            solutions.append(board[:])
            return
        for col in range(n):
            if is_safe(row, col):
                board[row] = col
                backtrack(row + 1)
                # No explicit undo needed (overwritten next iteration)
    
    backtrack(0)
    return solutions

# Display solutions
solutions = solve_n_queens(4)
for sol in solutions:
    print(sol)
```

**Output:**
```
[1, 3, 0, 2]
[2, 0, 3, 1]
```

**Time Complexity:** O(N!) worst-case, but pruning reduces it significantly.

### 20.5.2 Sudoku Solver

Fill a 9×9 grid with digits 1-9 so that each row, column, and 3×3 box contains all digits.

```python
def solve_sudoku(board):
    """board: list of list of ints, 0 represents empty cell."""
    def is_valid(board, row, col, num):
        # Check row
        for c in range(9):
            if board[row][c] == num:
                return False
        # Check column
        for r in range(9):
            if board[r][col] == num:
                return False
        # Check 3x3 box
        box_r, box_c = row // 3 * 3, col // 3 * 3
        for r in range(box_r, box_r + 3):
            for c in range(box_c, box_c + 3):
                if board[r][c] == num:
                    return False
        return True
    
    def backtrack():
        for r in range(9):
            for c in range(9):
                if board[r][c] == 0:
                    for num in range(1, 10):
                        if is_valid(board, r, c, num):
                            board[r][c] = num
                            if backtrack():
                                return True
                            board[r][c] = 0
                    return False
        return True  # no empty cells left
    
    backtrack()
    return board
```

### 20.5.3 Graph Coloring

Color vertices of a graph with at most m colors such that no adjacent vertices share the same color.

```python
def graph_coloring(graph, m):
    """graph: adjacency list, m: number of colors."""
    n = len(graph)
    color = [-1] * n
    
    def is_safe(v, c):
        for u in graph[v]:
            if color[u] == c:
                return False
        return True
    
    def backtrack(v):
        if v == n:
            return True
        for c in range(m):
            if is_safe(v, c):
                color[v] = c
                if backtrack(v + 1):
                    return True
                color[v] = -1
        return False
    
    if backtrack(0):
        return color
    return None
```

---

## 20.6 Pruning Strategies

**Pruning** eliminates branches that cannot lead to a solution, greatly improving efficiency.

### 20.6.1 Forward Checking

When assigning a variable, remove inconsistent values from domains of future variables.

### 20.6.2 Constraint Propagation (Arc Consistency)

Enforce consistency between variables: if a value for variable X is inconsistent with any value of Y, remove it.

### 20.6.3 Heuristics

- **Minimum Remaining Values (MRV):** Choose the variable with fewest remaining values.
- **Degree Heuristic:** Choose variable involved in most constraints.
- **Least Constraining Value:** Choose value that rules out fewest choices for neighbors.

Example: In N-Queens, we can prune by noting that no two queens share the same row (by design), column, or diagonal. Already implemented in `is_safe`.

---

## 20.7 Branch and Bound

**Branch and bound** is an optimization technique that explores the state space tree while maintaining bounds on the best possible solution. If a partial solution cannot improve the current best, it is pruned.

### 20.7.1 Example: 0/1 Knapsack

Given items with weights and values, maximize total value within weight capacity.

```python
def knapsack_branch_bound(items, capacity):
    """
    items: list of (value, weight)
    Returns max value and chosen items.
    """
    n = len(items)
    items = sorted(items, key=lambda x: x[0]/x[1], reverse=True)  # sort by value/weight
    best_value = 0
    best_taken = [0] * n
    
    def bound(i, current_weight, current_value):
        # Upper bound on achievable value from node i onward
        if current_weight >= capacity:
            return 0
        j = i
        total_value = current_value
        total_weight = current_weight
        while j < n and total_weight + items[j][1] <= capacity:
            total_weight += items[j][1]
            total_value += items[j][0]
            j += 1
        if j < n:
            # Fractional part of next item
            total_value += (capacity - total_weight) * items[j][0] / items[j][1]
        return total_value
    
    def backtrack(i, current_weight, current_value, taken):
        nonlocal best_value, best_taken
        if current_weight > capacity:
            return
        if current_value > best_value:
            best_value = current_value
            best_taken = taken[:]
        if i == n:
            return
        if bound(i, current_weight, current_value) <= best_value:
            return  # prune
        
        # Option 1: take item i
        if current_weight + items[i][1] <= capacity:
            taken[i] = 1
            backtrack(i+1, current_weight + items[i][1], current_value + items[i][0], taken)
            taken[i] = 0
        # Option 2: skip item i
        backtrack(i+1, current_weight, current_value, taken)
    
    backtrack(0, 0, 0, [0]*n)
    return best_value, best_taken
```

---

## 20.8 Meet-in-the-Middle

**Meet-in-the-Middle** splits the problem into two halves, solves each separately, and combines results. It reduces time from exponential to roughly square root of original.

### 20.8.1 Example: Subset Sum (exact target)

Given a set of integers, find if any subset sums to target T.

- Brute force: O(2ⁿ)
- Meet-in-the-middle: O(2ⁿ/²) time and space.

```python
def subset_sum_meet_in_middle(nums, target):
    n = len(nums)
    left = nums[:n//2]
    right = nums[n//2:]
    
    # Generate all subset sums for left
    left_sums = []
    for mask in range(1 << len(left)):
        s = sum(left[i] for i in range(len(left)) if mask & (1 << i))
        left_sums.append(s)
    
    # Generate all subset sums for right
    right_sums = []
    for mask in range(1 << len(right)):
        s = sum(right[i] for i in range(len(right)) if mask & (1 << i))
        right_sums.append(s)
    
    # Sort right_sums for binary search
    right_sums.sort()
    
    for s in left_sums:
        need = target - s
        # Binary search in right_sums
        lo, hi = 0, len(right_sums)
        while lo < hi:
            mid = (lo + hi) // 2
            if right_sums[mid] < need:
                lo = mid + 1
            else:
                hi = mid
        if lo < len(right_sums) and right_sums[lo] == need:
            return True
    return False
```

---

## 20.9 Applications of Backtracking

- **Puzzles:** N-Queens, Sudoku, crossword, KenKen
- **Combinatorial generation:** All permutations, combinations, subsets
- **Constraint satisfaction:** Scheduling, timetabling, resource allocation
- **Parsing:** Recursive descent parsers
- **Pathfinding in mazes:** DFS with backtracking
- **Configuration problems:** Graph coloring, Latin squares

---

## 20.10 Summary

```
┌─────────────────────────────────────────────────────────────────────┐
│                    RECURSION & BACKTRACKING SUMMARY                  │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Recursion: Function calls itself; base case + recursive case       │
│  Recursion tree: Visualizes recursive calls; aids complexity analysis│
│  Tail recursion: Recursive call is last operation; enables TCO      │
│                                                                      │
│  Backtracking: Incremental search with pruning of dead ends         │
│  State space tree: Nodes represent partial solutions                │
│                                                                      │
│  Pruning techniques:                                                │
│    • Forward checking                                               │
│    • Constraint propagation                                         │
│    • Heuristics (MRV, degree, LCV)                                 │
│    • Branch and bound                                               │
│                                                                      │
│  Meet-in-the-middle: Split problem, solve halves, combine           │
│                                                                      │
│  Classic problems: N-Queens, Sudoku, graph coloring, knapsack       │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 20.11 Practice Problems

### Problem 1: Generate All Subsets (LeetCode 78)
Given an array of distinct integers, return all possible subsets.

**Hint:** Use recursion with include/exclude at each step.

### Problem 2: Generate All Permutations (LeetCode 46)
Given an array of distinct integers, return all permutations.

**Hint:** Swap-based backtracking.

### Problem 3: Combination Sum (LeetCode 39)
Find all unique combinations of candidates that sum to target.

**Hint:** Backtrack with pruning if sum exceeds target.

### Problem 4: Word Search (LeetCode 79)
Given a 2D board and a word, find if word exists.

**Hint:** DFS from each cell with visited markers.

### Problem 5: Palindrome Partitioning (LeetCode 131)
Partition a string into substrings that are all palindromes.

**Hint:** Backtrack on cut positions; check palindrome.

### Problem 6: Sudoku Solver (LeetCode 37)
Implement a Sudoku solver.

**Hint:** Backtracking with constraint checks.

### Problem 7: N-Queens II (LeetCode 52)
Return number of distinct solutions for N-Queens.

**Hint:** Similar to N-Queens but only count.

### Problem 8: Split Array into Fibonacci Sequence (LeetCode 842)
Given a string of digits, split into Fibonacci sequence.

**Hint:** Backtrack with early pruning on overflow.

### Problem 9: Maximum Length of a Concatenated String with Unique Characters (LeetCode 1239)
Given a list of strings, find max length of concatenation with all unique chars.

**Hint:** Backtrack with bitmask representation for efficiency.

### Problem 10: Traveling Salesman Problem (TSP) – small instances
Find shortest tour visiting all cities exactly once. Use branch and bound with lower bound (MST heuristic).

---

## 20.12 Further Reading

1. **"Introduction to Algorithms" (CLRS)** – Chapter 4 (Divide-and-Conquer), Chapter 34 (NP-Completeness) for backtracking context.
2. **"The Algorithm Design Manual"** by Steven Skiena – Chapter 7 (Backtracking)
3. **"Artificial Intelligence: A Modern Approach"** by Russell & Norvig – Chapters on constraint satisfaction.
4. **"Programming Challenges"** by Skiena & Revilla – Many backtracking problems.
5. **Original Papers**:
   - Bitner, J. R., & Reingold, E. M. (1975) – "Backtrack programming techniques"
   - Golomb, S. W., & Baumert, L. D. (1965) – "Backtrack programming"

---

> **Coming in Chapter 21**: **Divide and Conquer** – We'll explore algorithms that break problems into independent subproblems, including merge sort, FFT, closest pair, and more.

---

**End of Chapter 20**